<a href="https://colab.research.google.com/github/JauraSeerat/Wonder_Vision_Face_Detection/blob/master/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

In [0]:
#Download Face evolve model files 

#Getting files require to run Face Evolve model based on MTCNN for Face Detection 

!wget https://raw.githubusercontent.com/ZhaoJ9014/face.evoLVe.PyTorch/master/align/align_trans.py -O ./align_trans.py
!wget https://raw.githubusercontent.com/ZhaoJ9014/face.evoLVe.PyTorch/master/align/box_utils.py -O ./box_utils.py
!wget https://raw.githubusercontent.com/ZhaoJ9014/face.evoLVe.PyTorch/master/align/detector.py -O ./detector.py
!wget https://raw.githubusercontent.com/ZhaoJ9014/face.evoLVe.PyTorch/master/align/face_align.py -O ./face_align.py
!wget https://raw.githubusercontent.com/ZhaoJ9014/face.evoLVe.PyTorch/master/align/face_resize.py -O ./face_resize.py
!wget https://raw.githubusercontent.com/ZhaoJ9014/face.evoLVe.PyTorch/master/align/first_stage.py -O ./first_stage.py
!wget https://raw.githubusercontent.com/ZhaoJ9014/face.evoLVe.PyTorch/master/align/get_nets.py -O ./get_nets.py
!wget https://raw.githubusercontent.com/ZhaoJ9014/face.evoLVe.PyTorch/master/align/matlab_cp2tform.py -O ./matlab.cpt2tform.py
  
#Getting the weights of three models Onet, Pnet and Rnet 
!wget https://github.com/ZhaoJ9014/face.evoLVe.PyTorch/raw/master/align/onet.npy -O ./onet.npy
!wget https://github.com/ZhaoJ9014/face.evoLVe.PyTorch/raw/master/align/pnet.npy -O ./pnet.npy
!wget https://github.com/ZhaoJ9014/face.evoLVe.PyTorch/raw/master/align/rnet.npy -O ./rnet.npy
  
!wget https://raw.githubusercontent.com/ZhaoJ9014/face.evoLVe.PyTorch/master/align/visualization_utils.py -O ./visualization_utils.py


In [0]:
#Main code 

from google.colab import files 

#Python Imports
from PIL import Image 
from detector import detect_faces
import matplotlib.pyplot as plt 
import time
import numpy as np 
import warnings
import torch
import torchvision.transforms as transforms
import random
import torch.nn as nn
import torch.nn.functional as F

warnings.filterwarnings('ignore')

#model_path = "/content/drive/My Drive//Colab//privateAI//Face//model//model.pt"
model_path = "/content/drive/My Drive//Colab//privateAI//Face//model//model.pt"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tensor_transform = transforms.ToTensor()

class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.cnn1 = nn.Sequential(
            nn.ReflectionPad2d(1),
            nn.Conv2d(1, 4, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(4),
            
            nn.ReflectionPad2d(1),
            nn.Conv2d(4, 8, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(8),


            nn.ReflectionPad2d(1),
            nn.Conv2d(8, 8, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(8),

        )

        self.fc1 = nn.Sequential(
            nn.Linear(8*100*100, 500),
            nn.ReLU(inplace=True),

            nn.Linear(500, 500),
            nn.ReLU(inplace=True),

            nn.Linear(500, 5))

    def forward_once(self, x):
        output = self.cnn1(x)
        output = output.view(output.size()[0], -1)
        output = self.fc1(output)
        return output

    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        return output1, output2
      
      
recognition_model = SiameseNetwork().to(device)
recognition_model.load_state_dict(torch.load(model_path))
recognition_model = recognition_model.to("cpu")
recognition_model = recognition_model.to(device)


def compare_faces(first_image,second_image):
  first_image = first_image.resize((100,100), Image.ANTIALIAS).convert('L')  
  second_image = second_image.resize((100,100), Image.ANTIALIAS).convert('L')   
  first_image_tensor = tensor_transform(first_image).unsqueeze(0).to(device)
  second_image_tensor = tensor_transform(second_image).unsqueeze(0).to(device)
  output_first,output_second = recognition_model(first_image_tensor,second_image_tensor)
  with torch.no_grad():
    similarity_score = 1 - F.pairwise_distance(output_first, output_second)  
  return similarity_score

def face_detection(image):
  bounding_box = detect_faces(image)
  cropped_image = image.crop((bounding_box[0][0][0],bounding_box[0][0][1],bounding_box[0][0][2],bounding_box[0][0][3]))
  return cropped_image


def get_similarity_score(first_image_path,second_image_path):
  first_image = Image.open(first_image_path).resize((255,255), Image.ANTIALIAS) 
  first_image_cropped = face_detection(first_image)
  second_image = Image.open(second_image_path).resize((255,255), Image.ANTIALIAS) 
  second_image_cropped = face_detection(second_image)
  similarity_score = compare_faces(first_image_cropped,second_image_cropped)
  similarity_score = similarity_score[0].to("cpu").numpy()
  return similarity_score,first_image,second_image

In [0]:
#@title Upload images
#Upload first image

print ('Hello User! Please upload first image')
uploaded = files.upload()
for fn in uploaded.keys():
  first_image_path = "./" + fn
  
  
print ('Please upload the second image')  
#Upload second image
uploaded = files.upload()
for fn in uploaded.keys():
  second_image_path = "./" + fn
  
  
# first_image_path = "./face3.jpg"  
# second_image_path = "./face4.jpg"
gridsize = (1,2)
fig = plt.figure(figsize=(10,10))
plt.subplots_adjust(wspace=0.5,hspace=0.05)

print ('Calculating the similarity score... Please wait for a moment')

sc,first_image,second_image = get_similarity_score(first_image_path,second_image_path)
print ('completed')


print ('\n\n\nSimilarity Score is %.2f' %(sc))
ax = plt.subplot2grid(gridsize, (0,0))
ax.grid(False)
ax.set_title("First Image")
ax.imshow(first_image)  
ax = plt.subplot2grid(gridsize, (0,1))
ax.grid(False)
ax.set_title("Second Image")
ax.imshow(second_image)  